In [21]:
import torchvision
from torchvision.io import read_image, ImageReadMode
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import Adam 
# from torchmetrics import Accuracy
from torchmetrics.functional import accuracy

In [22]:
# def get_data():
#     data_dir = '/your_path/Data/'

#     transform = transforms.Compose([
#         #transforms.RandomRotation(20),
#         transforms.RandomResizedCrop(128),
#         #transforms.RandomHorizontalFlip(),
#         transforms.ToTensor()])

#     train_set = datasets.ImageFolder(data_dir + '/training_set', transform=transform)
#     test_set = datasets.ImageFolder(data_dir + '/test_set', transform=transform)

#     train = DataLoader(train_set, batch_size=32, shuffle=True)
#     test = DataLoader(test_set, batch_size=32, shuffle=True)

#     return train, test

In [23]:
# dataset = torchvision.datasets.ImageFolder(root='images')

In [24]:
# transformations = transforms.Compose([
#     transforms.Resize((32,32)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

In [25]:
class data():
    def __init__(self, size, root, batch_size):
        self.size_ = size
        self.root_ = root
        self.batch_size_ = batch_size

    def prepare_dataset(self):
        transformations = transforms.Compose([
            transforms.Resize(self.size_),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        
        dataset = torchvision.datasets.ImageFolder(root=self.root_, transform=transformations)
        train_loader = DataLoader(dataset, batch_size = self.batch_size_, shuffle = True)
        print('Train set created !')
        return train_loader
        

In [95]:
class NeuralNetwork(nn.Module):
    def __init__(self, learning_rate, loss):
        super().__init__()                                               
        self.learning_rate_ = learning_rate
        self.loss_ = loss 

        self.cnn_ = nn.Sequential(
                        nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, padding=1),
                        nn.Conv2d(10,10,kernel_size=5, padding=1),
                        nn.MaxPool2d(2,2),
                        nn.Conv2d(in_channels=10, out_channels=10, kernel_size=5, padding=1),
                        nn.Conv2d(10,10,kernel_size=5, padding=1),
                        nn.MaxPool2d(2,2),
                        nn.Flatten(),
                        nn.Linear(5*5*10,2)
        )
        self.optimizer_ = Adam(self.cnn_.parameters(), lr=self.learning_rate_)

    def forward(self,x):
        return self.cnn_(x)
    
    def train(self, num_epoch, training_set):
        #best_accuracy = 0.0
        print("Training is beginning...")

        for epoch in range(1,num_epoch+1):
            true_class = torch.empty([0])
            predicted_class = torch.empty([0])
            for data in training_set:
                train_loss = 0.
                X,y = data 
                self.optimizer_.zero_grad()   # zero the parameter gradients          
                predicted_outputs = self.forward(X)   # predict output from the model 
                train_loss += self.loss_(predicted_outputs, y)   # calculate loss for the predicted output  
                train_loss.backward()   # backpropagate the loss 
                self.optimizer_.step()
                # print(predicted_outputs.data)
                # print(torch.max(predicted_outputs.data, 1)[1])
                # break
                predicted_class = torch.cat((predicted_class,torch.max(predicted_outputs, 1)[1]), dim=0)
                true_class = torch.cat((true_class,y), dim=0)
            print(f"Epoch {epoch} : Loss : {train_loss}, Accuracy : {accuracy(predicted_class, true_class, task='binary')}")

        print("Training is finished !")


In [27]:
data = data(size=32,
            root='images',
            batch_size=20)

train= data.prepare_dataset()

Train set created !


In [96]:
model = NeuralNetwork(learning_rate=1e-3,
                      loss=nn.CrossEntropyLoss())
model.train(5,train)

Training is beginning...
Epoch 1 : Loss : 0.555088996887207, Accuracy : 0.6219512224197388
Epoch 2 : Loss : 0.2530558109283447, Accuracy : 0.707317054271698
Epoch 3 : Loss : 1.228798270225525, Accuracy : 0.707317054271698
Epoch 4 : Loss : 0.429826021194458, Accuracy : 0.6829268336296082
Epoch 5 : Loss : 0.3912162780761719, Accuracy : 0.6951219439506531
Training is finished !


In [86]:
torch.cat((torch.empty([0]), torch.max(torch.Tensor(2,2), dim=1)[1]), dim=0)

tensor([0., 0.])

tensor([])